In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import torch
import numpy as np
import csv
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer,LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import math
import logging
from datetime import datetime
import os
import gzip
from scipy.stats import pearsonr
from tensorflow.keras.preprocessing.sequence import pad_sequences
from scipy.stats import pearsonr


In [4]:
os.chdir('drive/My Drive/NLP Assignment 3')

model = SentenceTransformer("Task1Cmodel")

In [12]:
df_test = pd.read_csv('/content/test_group_62.csv', delimiter='\t')         #!!change the directory according to the demo file
                                                                          #upload it here only

In [14]:
sentences_1 = df_test['sentence1'].tolist()
sentences_2 = df_test['sentence2'].tolist()

sentence_embeddings_1 = model.encode(sentences_1, convert_to_tensor=True)
sentence_embeddings_2 = model.encode(sentences_2, convert_to_tensor=True)

s = sentence_embeddings_1+sentence_embeddings_2
# if sent/
max_len = max(len(str(sent).split()) for sent in sentences_1 + sentences_2)

sentence_embeddings_1 = pad_sequences(sentence_embeddings_1, maxlen=max_len,dtype='float32', padding='post')
sentence_embeddings_2 = pad_sequences(sentence_embeddings_2, maxlen=max_len,dtype='float32', padding='post')


cos_sim = util.cos_sim(sentence_embeddings_1, sentence_embeddings_2)
scaled_cos_sim = 0.5 * (cos_sim + 1)
cos_sim = scaled_cos_sim

cosine_similarity = np.diag(cos_sim)
predictions = cosine_similarity    #score predictions

In [15]:
data = []
predictions = predictions*5
for i in range(len(sentences_1)):
  data.append([i+1,predictions[i],sentences_1[i],sentences_2[i]])

df_output = pd.DataFrame(data,columns = ['id','score','sentence1','sentence2'])

df_output.to_csv('output.csv',sep='\t',index = False)     #output.csv has been made in the drive
